In [1]:
import numpy as np
import pandas as pd
import pickle
import math
import os
import matplotlib.pyplot as plt
import collections
from scipy import stats
###################
n_f=7
n_node=365
###################
#label - load fact
y_area=np.load('data/data_cablearea.npy');print('y_area.shape',y_area.shape)
###node ddata 
#location feature
node_loc=(pd.read_excel('data/raw_data/node_location.xlsx',header=None).values)
#node_loc=np.array([node_loc]*1201);print('node_loc',node_loc.shape)
nf=np.load('data/nf.npy')
disp=np.load('data/disp.npy')#(12000, 365, 7)
#x=np.concatenate((disp,node_loc),axis=-1);
x=disp.copy();print(x.shape)#(1201, 365, 10)
#############################
features=x.copy()
label_area=y_area.copy()
#split
np.random.seed(1)
indices = np.random.permutation(len(features));print(indices[0])
n_training=int(len(features)*0.7)+4#6:1:3#7:1:2
n_val=int(len(features)*0.1)
n_test=len(features)-n_training-n_val
training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
f_training, f_val, f_test = features[training_idx],features[val_idx], features[test_idx]
print(f_training.shape,f_val.shape,f_test.shape)
l_training, l_val, l_test = label_area[training_idx],label_area[val_idx], label_area[test_idx]
print(l_training.shape,l_val.shape,l_test.shape)

#scale-features
'''m_list=[];std_list=[]
for i in range(n_f):
    m=np.mean(f_training[...,i].flatten());std=np.std(f_training[...,i].flatten())
    f_training[...,i]=(f_training[...,i]-m)/std
    f_val[...,i]=(f_val[...,i]-m)/std
    f_test[...,i]=(f_test[...,i]-m)/std
    m_list.append(m);std_list.append(std)
    print(i,np.min(f_val[...,i]),np.max(f_val[...,i]),np.mean(f_val[...,i]),np.std(f_val[...,i]))
m_list=np.array(m_list);std_list=np.array(std_list);
np.save('data/m_list',m_list)
np.save('data/std_list',std_list)'''
#scale-label
'''m=np.mean(l_training);std=np.std(l_training)
l_training=(l_training-m)/std
l_val=(l_val-m)/std
l_test=(l_test-m)/std
print('label m, std',m,",",std)'''

#kl divergenc loss data
l_training_kld=l_training/(np.sum(l_training,axis=1)[:,np.newaxis])
l_val_kld=l_val/(np.sum(l_val,axis=1)[:,np.newaxis])
l_test_kld=l_test/(np.sum(l_test,axis=1)[:,np.newaxis])
#multi task
def getting_multi(label_data,sf,mode):
    c_data=np.argmin(label_data,axis=1)
    a_data=np.min(label_data,axis=1)
    #c_data=c_data+1;c_data[a_data==1e-06]=0
    a_data=a_data[:,np.newaxis]
    if sf:
        np.savez_compressed('data/label_multitask/c_label_'+mode,c_data)
        np.savez_compressed('data/label_multitask/a_label_'+mode,a_data)
    return c_data,a_data

save_flag=True#False#
c_training,a_training=getting_multi(l_training,sf=save_flag,mode='train')
c_val,a_val=getting_multi(l_val,sf=save_flag,mode='val')
c_test,a_test=getting_multi(l_test,sf=save_flag,mode='test')

#displacement prediction
#f_training_6=f_training.copy();f_val_6=f_val.copy();f_test_6=f_test.copy()
#f_training_6[:,:-6,:7]=0;f_val_6[:,:-6,:7]=0;f_val_6[:,:-6,:7]=0;
f_training_6=f_training[:,-6:,:7];f_val_6=f_val[:,-6:,:7];f_test_6=f_test[:,-6:,:7];print(f_training_6.shape)
l_training_disp=f_training[:,:-6,:7].reshape((-1,359*7))
l_val_disp=f_val[:,:-6,:7].reshape((-1,359*7));
l_test_disp=f_test[:,:-6,:7].reshape((-1,359*7));

#add NF features
nf_training, nf_val, nf_test = nf[training_idx],nf[val_idx], nf[test_idx]
print(nf_training.shape,nf_val.shape,nf_test.shape)
m=np.mean(nf_training);std=np.std(nf_training)
nf_training=(nf_training-m)/std
nf_val=(nf_val-m)/std
nf_test=(nf_test-m)/std
print(nf_training.shape,nf_val.shape,nf_test.shape)
f_training_6_nf=np.concatenate((f_training_6.reshape((-1,6*7)),nf_training),axis=1)
f_val_6_nf=np.concatenate((f_val_6.reshape((-1,6*7)),nf_val),axis=1)
f_test_6_nf=np.concatenate((f_test_6.reshape((-1,6*7)),nf_test),axis=1)
print(f_training_6_nf.shape,f_val_6_nf.shape,f_test_6_nf.shape)

#save
#save_flag=False#True#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'features_train',f_training)
    np.savez_compressed(data_path+'features_val',f_val)
    np.savez_compressed(data_path+'features_test',f_test)
    data_path='data/'
    np.savez_compressed(data_path+'label_train',l_training)
    np.savez_compressed(data_path+'label_val',l_val)
    np.savez_compressed(data_path+'label_test',l_test)
    '''data_path='data/label_bclloss/'
    np.savez_compressed(data_path+'label_train',1-l_training)
    np.savez_compressed(data_path+'label_val',1-l_val)
    np.savez_compressed(data_path+'label_test',1-l_test)'''
    data_path='data/label_kld/'
    np.savez_compressed(data_path+'label_train',l_training_kld)
    np.savez_compressed(data_path+'label_val',l_val_kld)
    np.savez_compressed(data_path+'label_test',l_test_kld)
    
    data_path='data/label_disp/'
    np.savez_compressed(data_path+'label_train',l_training_disp)
    np.savez_compressed(data_path+'label_val',l_val_disp)
    np.savez_compressed(data_path+'label_test',l_test_disp)
    data_path='data/data_mpnn_6/'
    np.savez_compressed(data_path+'features_train',f_training_6)
    np.savez_compressed(data_path+'features_val',f_val_6)
    np.savez_compressed(data_path+'features_test',f_test_6)
    
    data_path='data/data_mlp/'
    np.savez_compressed(data_path+'features_train',f_training_6_nf)
    np.savez_compressed(data_path+'features_val',f_val_6_nf)
    np.savez_compressed(data_path+'features_test',f_test_6_nf)
    
# data_path='data/data_mlp/'
# np.save(data_path+'features_test_loc',f_test[:,:,7:])

y_area.shape (12000, 120)
(12000, 365, 7)
7163
(8404, 365, 7) (1200, 365, 7) (2396, 365, 7)
(8404, 120) (1200, 120) (2396, 120)
(8404, 6, 7)
(8404, 7) (1200, 7) (2396, 7)
(8404, 7) (1200, 7) (2396, 7)
(8404, 49) (1200, 49) (2396, 49)


In [1]:
import numpy as np
import pickle
import collections

adj_mx = np.load('data/sensor_graph/adj_mx.npy')
for i in range(len(adj_mx)): adj_mx[i,i]=0
print(adj_mx.shape,np.sum(adj_mx))
edge_data=np.array([[i,j] for i,j in zip(np.where(adj_mx==1)[0],np.where(adj_mx==1)[1])])

#distance 
edge_d=np.load('data/sensor_graph/dist_mx.npy')
for i in range(len(adj_mx)): edge_d[i,i]=0
edge_d=edge_d[edge_data[:,0],edge_data[:,1]]

#E 
edge_e=np.load('data/sensor_graph/e_mx.npy')
for i in range(len(adj_mx)): edge_e[i,i]=0
edge_e=edge_e[edge_data[:,0],edge_data[:,1]]

#area 
edge_a=np.load('data/sensor_graph/area_mx.npy')
for i in range(len(adj_mx)): edge_a[i,i]=0
edge_a=edge_a[edge_data[:,0],edge_data[:,1]]

#virtual
edge_v=np.load('data/sensor_graph/virtual_mx.npy')
for i in range(len(adj_mx)): edge_v[i,i]=0
edge_v=edge_v[edge_data[:,0],edge_data[:,1]]

#concatenate
edge_f=np.stack([edge_d,edge_e,edge_a,edge_v],axis=1)
print(edge_data.shape,edge_f.shape)

#self loop
edge_self=np.array([[i,i] for i in range(len(adj_mx))])
edge_data=np.concatenate([edge_data,edge_self],axis=0)
edge_f=np.concatenate([edge_f,np.zeros((len(adj_mx),edge_f.shape[1]))],axis=0)#matching shape (below)
edge_f_self=np.array([0]*(len(edge_f)-len(adj_mx))+[1]*(len(adj_mx)))[:,np.newaxis]#self loop feature
edge_f=np.concatenate([edge_f,edge_f_self],axis=1)
print(edge_data.shape,edge_f.shape)

#scale
for i in range(5):
    m=np.mean(edge_f[...,i].flatten());std=np.std(edge_f[...,i].flatten())
    edge_f[...,i]=(edge_f[...,i]-m)/std
    print(i,np.min(edge_f[...,i]),np.max(edge_f[...,i]),np.mean(edge_f[...,i]),np.std(edge_f[...,i]))

save_flag=True#False#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'edge_features',edge_f)
    np.savez_compressed(data_path+'edge_data',edge_data)
    
'''#split
np.random.seed(1)
indices = np.random.permutation(len(edge_f));print(indices[0])
n_training=int(len(edge_f)*0.7)+4#7:1:2
n_val=int(len(edge_f)*0.1)
n_test=len(edge_f)-n_training-n_val
training_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]
e_training, e_val,e_test = edge_f[training_idx],edge_f[val_idx], edge_f[test_idx]
print(e_training.shape,e_val.shape,e_test.shape)

#scale
for i in range(2):
    m=np.mean(e_training[...,i].flatten());std=np.std(e_training[...,i].flatten())
    e_training[...,i]=(e_training[...,i]-m)/std
    e_val[...,i]=(e_val[...,i]-m)/std
    e_test[...,i]=(e_test[...,i]-m)/std
    print(i,np.min(e_val[...,i]),np.max(e_val[...,i]),np.mean(e_val[...,i]),np.std(e_val[...,i]))
    
save_flag=False#True#
if save_flag:
    data_path='data/data_mpnn/'
    np.savez_compressed(data_path+'edge_train',e_training)
    np.savez_compressed(data_path+'edge_val',e_val)
    np.savez_compressed(data_path+'edge_test',e_test)
    np.savez_compressed(data_path+'edge_data',edge_data)'''

(365, 365) 972.0
(972, 2) (972, 4)
(1337, 2) (1337, 5)
0 -0.5041177487194609 3.4769474459377943 1.0628911529694842e-17 0.9999999999999999
1 -1.5951203569738215 0.725888881352981 -8.503129223755873e-17 1.0
2 -0.4679662821295938 3.845986154897304 -2.1257823059389683e-17 1.0
3 -0.09516619028617729 10.50793351076541 1.3286139412118552e-17 1.0
4 -0.6127923002879369 1.6318742900818486 8.503129223755873e-17 1.0


"#split\nnp.random.seed(1)\nindices = np.random.permutation(len(edge_f));print(indices[0])\nn_training=int(len(edge_f)*0.7)+4#7:1:2\nn_val=int(len(edge_f)*0.1)\nn_test=len(edge_f)-n_training-n_val\ntraining_idx, val_idx,test_idx = indices[:n_training], indices[n_training:n_training+n_val], indices[-n_test:]\ne_training, e_val,e_test = edge_f[training_idx],edge_f[val_idx], edge_f[test_idx]\nprint(e_training.shape,e_val.shape,e_test.shape)\n\n#scale\nfor i in range(2):\n    m=np.mean(e_training[...,i].flatten());std=np.std(e_training[...,i].flatten())\n    e_training[...,i]=(e_training[...,i]-m)/std\n    e_val[...,i]=(e_val[...,i]-m)/std\n    e_test[...,i]=(e_test[...,i]-m)/std\n    print(i,np.min(e_val[...,i]),np.max(e_val[...,i]),np.mean(e_val[...,i]),np.std(e_val[...,i]))\n    \nsave_flag=False#True#\nif save_flag:\n    data_path='data/data_mpnn/'\n    np.savez_compressed(data_path+'edge_train',e_training)\n    np.savez_compressed(data_path+'edge_val',e_val)\n    np.savez_compressed(d